In [30]:
# Import libraries
import cvxpy as cp
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# G = nx.Graph()
# G.add_nodes_from([0,8])
# G.add_weighted_edges_from([(0,5,1),(0,3,1),(3,4,1),(4,5,1),(4,6,1),(5,2,1),(3,1,1),(1,6,1),(6,2,1)])

# for n, nbrs in G.adj.items():
#     for nbr, eattr in nbrs.items():
#         wt = eattr['weight']
#         print(f"({n}, {nbr}, {wt})")


G = nx.DiGraph()
G.add_nodes_from([0,8])
G.add_edges_from([(6, 2), (3, 4), (4, 3), (3, 1), (6, 1), (5, 4), (0, 3), (4, 6), (6, 4), (3, 0), (4, 5), (5, 0), (2, 6), (0, 5), (1, 6), (2, 5), (1, 3), (5, 2)])

undirected_edges = set([(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)])

vertices = np.arange(9)

non_member = [3,4,5,6,7,8]

print(non_member)  # output: 3,4,5...8




source = 0
recievers = {1,2}

edges = set()

for n, nbrs in G.adj.items():
    for nbr, eattr in nbrs.items():
        edges.add((n,nbr))


print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []

c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]



fia = {(i, a): cp.Variable() for a in edges for i in recievers}
flow_constraints = []
for i in recievers: 
     for a in edges:
          flow_constraints += [fia[i,a] <= c[a]]
          flow_constraints += [fia[i,a] >= 0]


for i in recievers:
    for j in non_member:
        incoming_edges = [(u, v) for u, v in edges if v == int(j)]  # Filter edges based on the target node
        outcoming_edges = [(u, v) for u, v in edges if u == int(j)]  # Filter edges based on the source node
        incoming_flow = sum(fia[i, x] for x in incoming_edges)
        outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
        flow_constraints += [incoming_flow == outcoming_flow]

    incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  # Filter edges based on the source node
    
    outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i))  # Filter edges based on the target node
    flow_constraints += [incoming_flow_source == 0]
    flow_constraints += [outcoming_flow_reciever == 0]




# for i in recievers:
#      for j in non_member:
#           incoming_edges = G.in_edges(int(j))
#           outcoming_edges = G.out_edges(j)
#           incoming_flow = 0
#           outcoming_flow = 0
#           for x in incoming_edges:
#                incoming_flow = incoming_flow + fia[i,x]
#           for y in outcoming_edges:
#                outcoming_flow = outcoming_flow + fia[i,y]
#           flow_constraints += [incoming_flow == outcoming_flow]

#      incoming_flow_source = sum(fia[i, x] for x in G.in_edges(0))
#      outcoming_flow_reciever = sum(fia[i, x] for x in G.out_edges(i))
#      flow_constraints += [incoming_flow_source == 0]
#      flow_constraints += [outcoming_flow_reciever == 0]


     # incoming_edges_source = G.in_edges(j)
     # incoming_flow_source = 0
     # for x in incoming_edges_source:
     #      incoming_flow_source = incoming_flow_source + fia[i,x]
     # outcoming_edges = G.out_edges(j)    

    


equalrate_constraints = []

for i in recievers: 
     for a in edges:
          equalrate_constraints += [f_star == fia[i,a]]

objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)



# Solve the linear program
problem.solve()

# Display the results
print("Optimal Target Flow Rate (f*):", f_star.value)



[3, 4, 5, 6, 7, 8]
{(6, 2), (3, 4), (4, 3), (3, 1), (6, 1), (5, 4), (0, 3), (4, 6), (6, 4), (3, 0), (4, 5), (2, 6), (5, 0), (0, 5), (1, 6), (2, 5), (1, 3), (5, 2)}
Optimal Target Flow Rate (f*): 8.087955492607845e-13


In [ ]:
f_star = cp.Variable()  # Target flow rate


In [ ]:

# Define the network parameters (replace with your actual data)
num_nodes = 20  # Number of nodes
num_receivers = 5  # Number of multicast receivers
link_capacities = np.random.rand(num_nodes, num_nodes)  # Link capacities (random values for illustration)

# Create CVXPY variables
f_star = cp.Variable()  # Target flow rate


# Orientation constraints
C = cp.Variable((num_nodes, num_nodes), nonneg=True)  # Capacity matrix for orientation
orientation_constraints = [
    C == link_capacities,
    C >= 0
]

# Independent network flow constraints for each conceptual flow
fi = cp.Variable((num_receivers, num_nodes, num_nodes), nonneg=True)  # Flow matrix for each conceptual flow
network_flow_constraints = [
    fi[i, :, :] <= link_capacities,
    cp.sum(fi, axis=2) == 0,  # Flow conservation
    fi[:, 0, :] == 0,  # Incoming flow rate at the source
    fi[:, -1, :] == 0  # Outgoing flow rates at the receivers
]

# Equal rate constraints
equal_rate_constraints = [f_star == fi[i, i, -1] for i in range(num_receivers)]

# Formulate the linear program
objective = cp.Maximize(f_star)
constraints = orientation_constraints + network_flow_constraints + equal_rate_constraints
problem = cp.Problem(objective, constraints)

# Solve the linear program
problem.solve()

# Display the results
print("Optimal Target Flow Rate (f*):", f_star.value)

ValueError: Expressions of dimension greater than 2 are not supported.